In [1]:
import pandas as pd

school_path = 'Resources/schools_complete.csv'
student_path = 'Resources/students_complete.csv'

school_df = pd.read_csv(school_path)
student_df = pd.read_csv(student_path)

full_df = pd.merge(student_df, school_df, how='left', on=['school_name', 'school_name'])

# ----------------------------------
# District Summary DataFrame
# ----------------------------------

In [2]:
# Creating summary column variables
total_schools = len(full_df['school_name'].unique())
total_students = len(full_df['Student ID'].unique())
total_budget = full_df.budget.unique().sum()
average_math = (full_df.math_score.sum())/len(full_df.math_score) / 100
average_reading = (full_df.reading_score.sum())/len(full_df.reading_score) / 100
passing_math = (len(full_df.loc[full_df['math_score'] >= 70, :]) / total_students)
passing_reading = (len(full_df.loc[full_df['reading_score'] >= 70, :]) / total_students)

# Building district summary dataframe
district_raw = pd.DataFrame(
    {'Total Schools': total_schools,
     'Total Students': total_students, 
     'Total Budget': total_budget, 
     'Average Math Score': average_math, 
     'Average Reading Score': average_reading, 
     '% Passing Math': passing_math, 
     '% Passing Reading': passing_reading}, 
    index=[0])

# Formatting district summary dataframe
district_final = district_raw.style.format(
    {'Total Students': '{:,}',
     'Total Budget': '${:,.2f}',
     'Average Math Score': '{:.2%}', 
     'Average Reading Score': '{:.2%}', 
     '% Passing Math': '{:.2%}', 
     '% Passing Reading': '{:.2%}'})

# Printing district summary dataframe
district_final

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading
0,15,"39,170","$24,649,428.00",78.99%,81.88%,74.98%,85.81%


# ----------------------------------
# School Summary DataFrame
# ----------------------------------

In [3]:
# Creating .groupby object, grouped by school name
school_group = full_df.groupby('school_name')

# Building school summary dataframe using .groupby object
school_group_df = pd.DataFrame()
school_group_df['School Type'] = school_group['type'].first()
school_group_df['Total Students'] = school_group['Student ID'].count()
school_group_df['Total School Budget'] = school_group['budget'].max()
school_group_df['Per Student Budget'] = (school_group_df['Total School Budget'] / school_group_df['Total Students'])
school_group_df['Average Math Score'] = school_group['math_score'].mean() / 100
school_group_df['Average Reading Score'] = school_group['reading_score'].mean() / 100

# Building math score exclusive dataframe and creating .groupby to filter out passing grades
math_passing_df = full_df.loc[full_df['math_score'] >= 70, :]
math_passing_grouped = math_passing_df.groupby('school_name')
school_group_df['% Passing Math'] = (math_passing_grouped['Student ID'].count() / school_group_df['Total Students'])

# Building reading score exclusive dataframe and creating .groupby to filter out passing grades
reading_passing_df = full_df.loc[full_df['reading_score'] >= 70, :]
reading_passing_grouped = reading_passing_df.groupby('school_name')
school_group_df['% Passing Reading'] = (reading_passing_grouped['Student ID'].count() / school_group_df['Total Students'])

# Inserting overall rate column
school_group_df['% Overall Passing Rate'] = (school_group_df['% Passing Math'] + school_group_df['% Passing Reading'])/ 2

# Formatting school summary dataframe
school_summary_final = school_group_df.style.format(
    {'Total Students': '{:,}',
     'Total School Budget': '${:,.2f}',
     'Per Student Budget': '${:,.2f}',
     'Average Math Score': '{:.2%}', 
     'Average Reading Score': '{:.2%}', 
     '% Passing Math': '{:.2%}', 
     '% Passing Reading': '{:.2%}',
     '% Overall Passing Rate': '{:.2%}'})

# Printing school summary dataframe
school_summary_final

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05%,81.03%,66.68%,81.93%,74.31%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06%,83.98%,94.13%,97.04%,95.59%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71%,81.16%,65.99%,80.74%,73.36%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10%,80.75%,68.31%,79.30%,73.80%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35%,83.82%,93.39%,97.14%,95.27%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29%,80.93%,66.75%,80.86%,73.81%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80%,83.81%,92.51%,96.25%,94.38%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63%,81.18%,65.68%,81.32%,73.50%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07%,80.97%,66.06%,81.22%,73.64%


# --------------------------------------------------
# Top and Bottom Schools (by Passing Rate)
# --------------------------------------------------

## Top Schools by Passing Rate

In [4]:
# Building sorted top schools dataframe
top_schools_df = school_group_df.sort_values(by=['% Overall Passing Rate'], ascending=False)

# Formatting top schools dataframe
top_schools_final = top_schools_df.head(5).style.format(
    {'Total Students': '{:,}',
     'Total School Budget': '${:,.2f}',
     'Per Student Budget': '${:,.2f}',
     'Average Math Score': '{:.2%}', 
     'Average Reading Score': '{:.2%}', 
     '% Passing Math': '{:.2%}', 
     '% Passing Reading': '{:.2%}',
     '% Overall Passing Rate': '{:.2%}'})

# Printing top schools dataframe
top_schools_final

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06%,83.98%,94.13%,97.04%,95.59%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42%,83.85%,93.27%,97.31%,95.29%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84%,84.04%,94.59%,95.95%,95.27%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35%,83.82%,93.39%,97.14%,95.27%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27%,83.99%,93.87%,96.54%,95.20%


## Bottom Schools by Passing Rate

In [5]:
# Building sorted bottom schools dataframe
bottom_schools_df = school_group_df.sort_values(by=['% Overall Passing Rate'])

# Formatting bottom schools dataframe
bottom_schools_final = bottom_schools_df.head(5).style.format(
    {'Total Students': '{:,}',
     'Total School Budget': '${:,.2f}',
     'Per Student Budget': '${:,.2f}',
     'Average Math Score': '{:.2%}', 
     'Average Reading Score': '{:.2%}', 
     '% Passing Math': '{:.2%}', 
     '% Passing Reading': '{:.2%}',
     '% Overall Passing Rate': '{:.2%}'})

# Printing bottom schools dataframe
bottom_schools_final

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84%,80.74%,66.37%,80.22%,73.29%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71%,81.16%,65.99%,80.74%,73.36%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63%,81.18%,65.68%,81.32%,73.50%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07%,80.97%,66.06%,81.22%,73.64%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10%,80.75%,68.31%,79.30%,73.80%


# -------------------------------------------
# Math and Reading Scores (by Grade)
# -------------------------------------------

## Math

In [6]:
# Building grade exclusive dataframes and .groupby objects by school name
ninth_math = full_df.loc[full_df.grade == '9th', ['school_name', 'math_score']]
ninth_math_group = ninth_math.groupby('school_name')
tenth_math = full_df.loc[full_df.grade == '10th', ['school_name', 'math_score']]
tenth_math_group = tenth_math.groupby('school_name')
eleventh_math = full_df.loc[full_df.grade == '11th', ['school_name', 'math_score']]
eleventh_math_group = eleventh_math.groupby('school_name')
twelfth_math = full_df.loc[full_df.grade == '12th', ['school_name', 'math_score']]
twelfth_math_group = twelfth_math.groupby('school_name')

# Building grade sorted summary dataframe
math_by_grade_df = pd.DataFrame()
math_by_grade_df['9th'] = ninth_math_group['math_score'].mean() / 100
math_by_grade_df['10th'] = tenth_math_group['math_score'].mean() / 100
math_by_grade_df['11th'] = eleventh_math_group['math_score'].mean() / 100
math_by_grade_df['12th'] = twelfth_math_group['math_score'].mean() / 100

# Formatting grade sorted summary dataframe
math_by_grade_final = math_by_grade_df.style.format(
    {'9th': '{:.2%}',
     '10th': '{:.2%}',
     '11th': '{:.2%}',
     '12th': '{:.2%}'})

# Printing grade sorted summary dataframe
math_by_grade_final

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08%,77.00%,77.52%,76.49%
Cabrera High School,83.09%,83.15%,82.77%,83.28%
Figueroa High School,76.40%,76.54%,76.88%,77.15%
Ford High School,77.36%,77.67%,76.92%,76.18%
Griffin High School,82.04%,84.23%,83.84%,83.36%
Hernandez High School,77.44%,77.34%,77.14%,77.19%
Holden High School,83.79%,83.43%,85.00%,82.86%
Huang High School,77.03%,75.91%,76.45%,77.23%
Johnson High School,77.19%,76.69%,77.49%,76.86%


## Reading

In [7]:
# Building grade exclusive dataframes and .groupby objects by school name
ninth_reading = full_df.loc[full_df.grade == '9th', ['school_name', 'reading_score']]
ninth_reading_group = ninth_reading.groupby('school_name')
tenth_reading = full_df.loc[full_df.grade == '10th', ['school_name', 'reading_score']]
tenth_reading_group = tenth_reading.groupby('school_name')
eleventh_reading = full_df.loc[full_df.grade == '11th', ['school_name', 'reading_score']]
eleventh_reading_group = eleventh_reading.groupby('school_name')
twelfth_reading = full_df.loc[full_df.grade == '12th', ['school_name', 'reading_score']]
twelfth_reading_group = twelfth_reading.groupby('school_name')

# Building grade sorted summary dataframe
reading_by_grade_df = pd.DataFrame()
reading_by_grade_df['9th'] = ninth_reading_group['reading_score'].mean() / 100
reading_by_grade_df['10th'] = tenth_reading_group['reading_score'].mean() / 100
reading_by_grade_df['11th'] = eleventh_reading_group['reading_score'].mean() / 100
reading_by_grade_df['12th'] = twelfth_reading_group['reading_score'].mean() / 100

# Formatting grade sorted summary dataframe
reading_by_grade_final = reading_by_grade_df.style.format(
    {'9th': '{:.2%}',
     '10th': '{:.2%}',
     '11th': '{:.2%}',
     '12th': '{:.2%}'})

# Printing grade sorted summary dataframe
reading_by_grade_final

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30%,80.91%,80.95%,80.91%
Cabrera High School,83.68%,84.25%,83.79%,84.29%
Figueroa High School,81.20%,81.41%,80.64%,81.38%
Ford High School,80.63%,81.26%,80.40%,80.66%
Griffin High School,83.37%,83.71%,84.29%,84.01%
Hernandez High School,80.87%,80.66%,81.40%,80.86%
Holden High School,83.68%,83.32%,83.82%,84.70%
Huang High School,81.29%,81.51%,81.42%,80.31%
Johnson High School,81.26%,80.77%,80.62%,81.23%


# ----------------------------------------
# Total Scores (by School Spending)
# ----------------------------------------

In [8]:
# Initializing school spending bins and labels
school_spending_bins = [0, 580, 605, 630, 655]
school_spending_labels = ['Less than $580', '$580-604', '$605-629', '$630-655']

# Inserting bin-controlled 'Spending Ranges' column into dataframe, then creating .groupby object using new column
school_group_df['Spending Ranges (Per Student)'] = pd.cut(school_group_df['Per Student Budget'], school_spending_bins, labels=school_spending_labels, include_lowest=True)
spending_bins_grouped = school_group_df.groupby('Spending Ranges (Per Student)')

# Building school spending summary dataframe using spending groups
school_spending_df = pd.DataFrame()
school_spending_df['Average Math Score'] = spending_bins_grouped['Average Math Score'].mean()
school_spending_df['Average Reading Score'] = spending_bins_grouped['Average Reading Score'].mean()
school_spending_df['% Passing Math'] = spending_bins_grouped['% Passing Math'].mean()
school_spending_df['% Passing Reading'] = spending_bins_grouped['% Passing Reading'].mean()
school_spending_df['% Overall Passing Rate'] = spending_bins_grouped['% Overall Passing Rate'].mean()

# Formatting school spending summary dataframe
school_spending_final = school_spending_df.style.format(
    {'Average Math Score': '{:.2%}', 
     'Average Reading Score': '{:.2%}', 
     '% Passing Math': '{:.2%}', 
     '% Passing Reading': '{:.2%}',
     '% Overall Passing Rate': '{:.2%}'})

# Printing spending summary dataframe
school_spending_final

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
Less than $580,83.27%,83.99%,93.87%,96.54%,95.20%
$580-604,83.48%,83.87%,93.46%,96.44%,94.95%
$605-629,81.41%,82.97%,84.89%,91.67%,88.28%
$630-655,77.87%,81.37%,70.35%,83.00%,76.67%


# ----------------------------------
# Total Scores (by School Size)
# ----------------------------------

In [9]:
# Initializing school size bins and labels
school_size_bins = [0, 1000, 2000, 3000, 4000, 5000]
school_size_labels = ['0-999', '1000-1999', '2000-2999', '3000-3999', '4000-4999']

# Inserting bin-controlled 'School Size' column into dataframe, then creating .groupby object using new column
school_group_df['School Size'] = pd.cut(school_group_df['Total Students'], school_size_bins, labels=school_size_labels, include_lowest=True)
school_size_grouped = school_group_df.groupby('School Size')

# Building school size summary dataframe using size groups
school_size_df = pd.DataFrame()
school_size_df['Average Math Score'] = school_size_grouped['Average Math Score'].mean()
school_size_df['Average Reading Score'] = school_size_grouped['Average Reading Score'].mean()
school_size_df['% Passing Math'] = school_size_grouped['% Passing Math'].mean()
school_size_df['% Passing Reading'] = school_size_grouped['% Passing Reading'].mean()
school_size_df['% Overall Passing Rate'] = school_size_grouped['% Overall Passing Rate'].mean()

# Formatting school size summary dataframe
school_size_final = school_size_df.style.format(
    {'Average Math Score': '{:.2%}', 
     'Average Reading Score': '{:.2%}', 
     '% Passing Math': '{:.2%}', 
     '% Passing Reading': '{:.2%}',
     '% Overall Passing Rate': '{:.2%}'})

# Printing school size dataframe
school_size_final

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
0-999,83.82%,83.93%,93.55%,96.10%,94.82%
1000-1999,83.37%,83.86%,93.60%,96.79%,95.20%
2000-2999,78.43%,81.77%,73.46%,84.47%,78.97%
3000-3999,76.84%,80.74%,66.37%,80.22%,73.29%
4000-4999,77.14%,80.98%,66.50%,81.34%,73.92%


# ----------------------------------
# Total Scores (by School Type)
# ----------------------------------

In [10]:
# Creating .groupby object by school type
school_type_grouped = school_group_df.groupby('School Type')

# Building school type summary dataframe
school_type_df = pd.DataFrame()
school_type_df['Average Math Score'] = school_type_grouped['Average Math Score'].mean()
school_type_df['Average Reading Score'] = school_type_grouped['Average Reading Score'].mean()
school_type_df['% Passing Math'] = school_type_grouped['% Passing Math'].mean()
school_type_df['% Passing Reading'] = school_type_grouped['% Passing Reading'].mean()
school_type_df['% Overall Passing Rate'] = school_type_grouped['% Overall Passing Rate'].mean()

# Formatting school type summary dataframe
school_type_final = school_type_df.style.format(
    {'Average Math Score': '{:.2%}', 
     'Average Reading Score': '{:.2%}', 
     '% Passing Math': '{:.2%}', 
     '% Passing Reading': '{:.2%}',
     '% Overall Passing Rate': '{:.2%}'})

# Printing school type summary dataframe
school_type_final

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.47%,83.90%,93.62%,96.59%,95.10%
District,76.96%,80.97%,66.55%,80.80%,73.67%
